In [2]:
import numpy as np
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
# Загрузка данных
df = pd.read_csv('data/ml-latest-small/ratings.csv')
n_users = df['userId'].nunique()
n_items = df['movieId'].nunique()

# Создание индексов для пользователей и фильмов
user_ids = df['userId'].astype('category').cat.codes.values
item_ids = df['movieId'].astype('category').cat.codes.values

df['user_idx'] = user_ids
df['item_idx'] = item_ids

# Разделение данных на тренировочные и тестовые наборы
train_df, test_df = train_test_split(df, test_size=0.7, random_state=42)

# Матрица рейтингов
def create_matrix(df, n_users, n_items):
    matrix = np.zeros((n_users, n_items))
    for row in df.itertuples():
        matrix[row.user_idx, row.item_idx] = row.rating
    return matrix

R_train = create_matrix(train_df, n_users, n_items)
R_test = create_matrix(test_df, n_users, n_items)

In [4]:
# Функция для расчета RMSE
def calculate_rmse(R_test, model):
    xs, ys = R_test.nonzero()
    predicted = []
    actual = []
    # progress_bar = tqdm(total=len(xs), desc="Calculating RMSE")
    for x, y in zip(xs, ys):
        predicted.append(model.predict(x, y))
        actual.append(R_test[x, y])
        # progress_bar.update(1)
    # progress_bar.close()
    return np.sqrt(np.mean((np.array(predicted) - np.array(actual)) ** 2))

# Функция для расчета precision@k
def precision_at_k(recommended_items, relevant_items, k):
    recommended_k = recommended_items[:k]
    hits = len(set(recommended_k) & set(relevant_items))
    return hits / k

# Функция для расчета Average Precision (AP) для одного пользователя
def average_precision_at_k(recommended_items, relevant_items, k):
    # Проверяем, есть ли релевантные элементы
    if len(relevant_items) == 0:
        return 0.0
    
    score = 0.0
    num_hits = 0.0
    for i in range(1, k+1):
        if recommended_items[i-1] in relevant_items:
            num_hits += 1.0
            score += num_hits / i
    return score / min(len(relevant_items), k)

# Функция для расчета MAP@50 для всех пользователей
def mean_average_precision_at_50(model, R_test, top_k=50):
    aps = []
    num_users = R_test.shape[0]
    for user in range(num_users):
        # Получаем релевантные айтемы из тестового набора (например, рейтинги >= 4)
        relevant_items = np.where(R_test[user, :] >= 4)[0]
        if len(relevant_items) == 0:
            continue  # Пропускаем пользователей без релевантных айтемов в тесте

        # Получаем предсказанные рейтинги для всех айтемов
        scores = model.predict(user, np.arange(model.num_items))
        # Сортируем айтемы по убыванию предсказанных рейтингов
        recommended_items = np.argsort(-scores)

        # Вычисляем Average Precision для текущего пользователя
        ap = average_precision_at_k(recommended_items, relevant_items, top_k)
        aps.append(ap)

    return np.mean(aps)

In [20]:
class SVD:
    def __init__(self, R, K=20, alpha=0.002, beta=0.02, iterations=100):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha  # Скорость обучения
        self.beta = beta  # Регуляризация
        self.iterations = iterations
        self.rmse_values = []  # Список для хранения значений RMSE
        self.map_values = []  # Список для хранения значений MAP


    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R > 0])

        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        progress_bar = tqdm(total=self.iterations, desc="Training SVD")
        for iteration in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            map50 = mean_average_precision_at_50(self, R_test, top_k=50)
            self.rmse_values.append(rmse)  # Сохраняем значение RMSE
            self.map_values.append(map50)  # Сохраняем значение MAP
            progress_bar.set_postfix({'RMSE': f"{rmse:.4f}", 'MAP': f"{map50:.4f}"})
            progress_bar.update(1)
        # progress_bar.close()

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.predict(i, j)
            error = r - prediction

            # Обновление биасов
            self.b_u[i] += self.alpha * (error - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (error - self.beta * self.b_i[j])

            # Обновление скрытых факторов
            self.P[i, :] += self.alpha * (error * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (error * self.P[i, :] - self.beta * self.Q[j, :])


    def predict(self, i, j):
        return self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)


    def rmse(self):
        xs, ys = self.R.nonzero()
        predicted = []
        actual = []
        for x, y in zip(xs, ys):
            predicted.append(self.predict(x, y))
            actual.append(self.R[x, y])
        return np.sqrt(np.mean((np.array(predicted) - np.array(actual)) ** 2))


    def plot_results(self):
        fig, ax1 = plt.subplots()
        
        # Первая ось y (RMSE)
        ax1.plot(self.rmse_values, label='RMSE', color='b')
        ax1.set_xlabel('Итерация')
        ax1.set_ylabel('RMSE', color='b')
        ax1.tick_params(axis='y', labelcolor='b')
        
        # Второстепенная ось y (MAP)
        ax2 = ax1.twinx()
        ax2.plot(self.map_values, label='MAP', color='r')
        ax2.set_ylabel('MAP', color='r')
        ax2.tick_params(axis='y', labelcolor='r')
        
        # Общая легенда
        lines = ax1.get_lines() + ax2.get_lines()
        labels = [line.get_label() for line in lines]
        plt.legend(lines, labels, loc='upper left')
        
        plt.title('Зависимость RMSE и MAP от итерации')
        plt.show()

In [21]:
class SVDPlusPlus:
    def __init__(self, R, K=20, alpha=0.002, beta=0.02, iterations=100):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K  # Количество скрытых факторов
        self.alpha = alpha  # Скорость обучения
        self.beta = beta  # Регуляризация
        self.iterations = iterations
        self.rmse_values = []  # Список для хранения значений RMSE
        self.map_values = []  # Список для хранения значений MAP

        # Инициализация параметров
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))  # Пользовательские факторы
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))  # Товарные факторы
        self.y = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))  # Латентные векторы для товаров

        self.b_u = np.zeros(self.num_users)  # Биасы пользователей
        self.b_i = np.zeros(self.num_items)  # Биасы товаров
        self.b = np.mean(self.R[self.R > 0])  # Глобальный биас

        # Предварительное сохранение индексов непустых оценок
        self.samples = [
            (int(i), int(j), self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Предварительное вычисление множества N_u для каждого пользователя
        self.N_u = {i: np.where(self.R[i, :] > 0)[0] for i in range(self.num_users)}

    def train(self):
        progress_bar = tqdm(total=self.iterations, desc="Training SVD++")
        for iteration in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse = self.rmse()
            map50 = mean_average_precision_at_50(self, R_test, top_k=50)
            self.rmse_values.append(rmse)  # Сохраняем значение RMSE
            self.map_values.append(map50)  # Сохраняем значение MAP
            progress_bar.set_postfix({'RMSE': f"{rmse:.4f}", 'MAP': f"{map50:.4f}"})
            progress_bar.update(1)
        # progress_bar.close()

    def sgd(self):
        y_update = np.zeros_like(self.y)  # Для обновления латентных векторов y_j

        for i, j, r in self.samples:
            # Имплицитный фидбек для пользователя i
            N_u = self.N_u[i]
            sqrt_len_N_u = np.sqrt(len(N_u)) if len(N_u) > 0 else 1  # Число sqrt(len(N_u))
            implicit_feedback = np.sum(self.y[N_u], axis=0) / sqrt_len_N_u if len(N_u) > 0 else np.zeros(self.K)

            # Предсказание
            prediction = self.b + self.b_u[i] + self.b_i[j] + np.dot(self.P[i, :] + implicit_feedback, self.Q[j, :].T)
            error = r - prediction

            # Обновление биасов
            self.b_u[i] += self.alpha * (error - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (error - self.beta * self.b_i[j])

            # Обновление факторов
            self.P[i, :] += self.alpha * (error * self.Q[j, :] - self.beta * self.P[i, :])
            self.Q[j, :] += self.alpha * (error * (self.P[i, :] + implicit_feedback) - self.beta * self.Q[j, :])

            # Обновление латентных факторов y для каждого товара в N_u
            if len(N_u) > 0:
                y_update[N_u] += self.alpha * (error / sqrt_len_N_u * self.Q[j, :] - self.beta * self.y[N_u])

        # Обновление латентных векторов y для всех товаров после прохода по данным
        self.y += y_update

    def predict(self, i, j):
        N_u = self.N_u[i]
        sqrt_len_N_u = np.sqrt(len(N_u)) if len(N_u) > 0 else 1
        implicit_feedback = np.sum(self.y[N_u], axis=0) / sqrt_len_N_u if len(N_u) > 0 else np.zeros(self.K)
        return self.b + self.b_u[i] + self.b_i[j] + np.dot(self.P[i, :] + implicit_feedback, self.Q[j, :].T)

    def rmse(self):
        xs, ys = self.R.nonzero()
        predicted = np.array([self.predict(x, y) for x, y in zip(xs, ys)])
        actual = self.R[xs, ys]
        return np.sqrt(np.mean((predicted - actual) ** 2))

    def plot_results(self):
        fig, ax1 = plt.subplots()
        
        # Первая ось y (RMSE)
        ax1.plot(self.rmse_values, label='RMSE', color='b')
        ax1.set_xlabel('Итерация')
        ax1.set_ylabel('RMSE', color='b')
        ax1.tick_params(axis='y', labelcolor='b')
        
        # Второстепенная ось y (MAP)
        ax2 = ax1.twinx()
        ax2.plot(self.map_values, label='MAP', color='r')
        ax2.set_ylabel('MAP', color='r')
        ax2.tick_params(axis='y', labelcolor='r')
        
        # Общая легенда
        lines = ax1.get_lines() + ax2.get_lines()
        labels = [line.get_label() for line in lines]
        plt.legend(lines, labels, loc='upper left')
        
        plt.title('Зависимость RMSE и MAP от итерации')
        plt.show()

In [6]:
param_grid = {
    'K': [2, 10, 50, 100, 200, 400, 600],
    'iterations': [10],
    'alpha': [0.001],
    'beta': [0.02]
}

results = []

for K in param_grid['K']:
    for iterations in param_grid['iterations']:
        for alpha in param_grid['alpha']:
            for beta in param_grid['beta']:
                # Запуск моделей
                svd = SVD(R_train, K=K, iterations=iterations, alpha=alpha, beta=beta)
                svd.train()
                svd_rmse = calculate_rmse(R_test, svd)
                svd_map = svd.map_values[-1]
                
                # svdpp = SVDPlusPlus(R_train, K=K, iterations=iterations, alpha=alpha, beta=beta)
                # svdpp.train()
                # svdpp_rmse = svdpp.rmse_values[-1]
                # svdpp_map = svdpp.map_values[-1]
                
                results.append([K, iterations, alpha, beta, svd_rmse, svd_map])

df = pd.DataFrame(results, columns=['K', 'Итерации', 'Alpha', 'Beta', 'SVD RMSE', 'SVD MAP'])

print(tabulate(df, headers='keys', tablefmt='psql'))

Training SVD: 100%|██████████| 10/10 [02:28<00:00, 14.88s/it, RMSE=0.9297, MAP=0.0904]


+----+-----+------------+---------+--------+------------+-----------+
|    |   K |   Итерации |   Alpha |   Beta |   SVD RMSE |   SVD MAP |
|----+-----+------------+---------+--------+------------+-----------|
|  0 |   2 |         10 |   0.001 |   0.02 |   0.972913 | 0.0193098 |
|  1 |  10 |         10 |   0.001 |   0.02 |   0.942912 | 0.086041  |
|  2 |  50 |         10 |   0.001 |   0.02 |   0.942302 | 0.090358  |
|  3 | 100 |         10 |   0.001 |   0.02 |   0.942318 | 0.0903877 |
|  4 | 200 |         10 |   0.001 |   0.02 |   0.942324 | 0.0904002 |
|  5 | 400 |         10 |   0.001 |   0.02 |   0.942312 | 0.0904082 |
|  6 | 600 |         10 |   0.001 |   0.02 |   0.94232  | 0.0903813 |
+----+-----+------------+---------+--------+------------+-----------+


**Вывод 1:**
1. В качестве данных для обучения взято 30%. 70% использовано для расчета MAP@50. Это связано с тем, что малая выборка для теста дает низкий MAP сам по себе, а если взять весь набор данных, то результаты лучше, так как SVD научен на тренировочных данных и их же предсказывает.
2. Скрытая размерность более 50 не нужна, так как модель точнее не становится, а время обучения растет.
3. Использована для подбора _K_ только SVD (не SVD++), так как скорость обучения у нее выше, а результаты с SVD++ похожи.

In [23]:
param_grid = {
    'K': [50],
    'iterations': [10, 15, 20],
    'alpha': [0.001, 0.003],
    'beta': [0.02, 0.1, 1]
}

results = []

for K in param_grid['K']:
    for iterations in param_grid['iterations']:
        for alpha in param_grid['alpha']:
            for beta in param_grid['beta']:
                # Запуск моделей
                svd = SVD(R_train, K=K, iterations=iterations, alpha=alpha, beta=beta)
                svd.train()
                svd_rmse = calculate_rmse(R_test, svd)
                svd_map = svd.map_values[-1]
                
                svdpp = SVDPlusPlus(R_train, K=K, iterations=iterations, alpha=alpha, beta=beta)
                svdpp.train()
                svdpp_rmse = calculate_rmse(R_test, svdpp)
                svdpp_map = svdpp.map_values[-1]
                
                results.append([K, iterations, alpha, beta, svd_rmse, svd_map, svdpp_rmse, svdpp_map])

df = pd.DataFrame(results, columns=['K', 'Итерации', 'Alpha', 'Beta', 'SVD RMSE', 'SVD MAP', 'SVD++ RMSE', 'SVD++ MAP'])

print(tabulate(df, headers='keys', tablefmt='psql'))































































































































Training SVD: 100%|██████████| 10/10 [00:26<00:00,  2.61s/it, RMSE=0.9293, MAP=0.0904]






























































































































Training SVD++: 100%|██████████| 10/10 [01:09<00:00,  6.99s/it, RMSE=0.9290, MAP=0.0900]






























































































































Training SVD: 100%|██████████| 10/10 [00:25<00:00,  2.55s/it, RMSE=0.9308, MAP=0.0902]












































In [7]:
# K = 2
# iterations = 10
# alpha = 0.003
# beta = 0.02

# # Запуск моделей
# svd = SVD(R_train, K=K, iterations=iterations, alpha=alpha, beta=beta)
# svd.train()
# svd.plot_results()

# svdpp = SVDPlusPlus(R_train, K=K, iterations=iterations, alpha=alpha, beta=beta)
# svdpp.train()
# svdpp.plot_results()

In [8]:
# map_50 = mean_average_precision_at_50(svd, R_test, top_k=50)
# print(f"MAP@50: {map_50:.4f}")

# map_50 = mean_average_precision_at_50(svdpp, R_test, top_k=50)
# print(f"MAP@50: {map_50:.4f}")